In [ ]:
!pip install keras

In [ ]:
import string
import numpy as np
import os
from pickle import dump, load
import numpy as np
import tensorflow as tf

In [ ]:

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm_notebook as tqdm

In [ ]:
dataset_path = "/content/seu_arquivo_filtrado.json"
text = open(dataset_path, 'rb').read().decode(encoding='utf-8')
text

'[\n    {\n        "source_file": "Benchmark Questions Verification V2.ipynb",\n        "task_id": 2,\n        "test_imports": [],\n        "test_list": [\n            "assert set(similar_elements((3, 4, 5, 6),(5, 7, 4, 10))) == set((4, 5))",\n            "assert set(similar_elements((1, 2, 3, 4),(5, 4, 3, 7))) == set((3, 4))",\n            "assert set(similar_elements((11, 12, 14, 13),(17, 15, 14, 13))) == set((13, 14))"\n        ],\n        "prompt_code": "Prompt: Write a function to find the shared elements from the given two lists. | Code: def similar_elements(test_tup1, test_tup2):\\n  res = tuple(set(test_tup1) & set(test_tup2))\\n  return (res) "\n    },\n    {\n        "source_file": "Benchmark Questions Verification V2.ipynb",\n        "task_id": 3,\n        "test_imports": [],\n        "test_list": [\n            "assert is_not_prime(2) == False",\n            "assert is_not_prime(10) == True",\n            "assert is_not_prime(35) == True",\n            "assert is_not_prime(

In [ ]:
sentences = text.splitlines()

In [ ]:
sentences

['[',
 '    {',
 '        "source_file": "Benchmark Questions Verification V2.ipynb",',
 '        "task_id": 2,',
 '        "test_imports": [],',
 '        "test_list": [',
 '            "assert set(similar_elements((3, 4, 5, 6),(5, 7, 4, 10))) == set((4, 5))",',
 '            "assert set(similar_elements((1, 2, 3, 4),(5, 4, 3, 7))) == set((3, 4))",',
 '            "assert set(similar_elements((11, 12, 14, 13),(17, 15, 14, 13))) == set((13, 14))"',
 '        ],',
 '        "prompt_code": "Prompt: Write a function to find the shared elements from the given two lists. | Code: def similar_elements(test_tup1, test_tup2):\\n  res = tuple(set(test_tup1) & set(test_tup2))\\n  return (res) "',
 '    },',
 '    {',
 '        "source_file": "Benchmark Questions Verification V2.ipynb",',
 '        "task_id": 3,',
 '        "test_imports": [],',
 '        "test_list": [',
 '            "assert is_not_prime(2) == False",',
 '            "assert is_not_prime(10) == True",',
 '            "assert is_

In [ ]:
def text_vocabulary(captions_list):
  vocab = set()
  for caption in captions_list:
    vocab.update(caption.split())
  return vocab

In [ ]:
sentences_with_tokens = []

def add_start_end_tokens(sentences):
    for sentence in sentences:
      sentence_with_tokens = '<start> ' + sentence + ' <end>'
    sentences_with_tokens.append(sentence_with_tokens)
    return sentences_with_tokens

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

def create_tokenizer(sentences_with_tokens):
 tokenizer = Tokenizer()
 tokenizer.fit_on_texts(sentences_with_tokens)
 return tokenizer

In [ ]:

def max_length(sentences_with_tokens):
 return max(len(d.split()) for d in sentences_with_tokens)



In [ ]:
import json

# Carregar o arquivo JSON
with open('/content/seu_arquivo_filtrado.json', 'r') as file:
    dados_json = json.load(file)



todos_textos = []


for item in dados_json:
    if 'prompt_code' in item:
        # Juntar o 'prompt' e o 'code' de cada item em uma única string
        prompt_code_str = " | ".join(item['prompt_code'])  # Usando um delimitador ' | '
        todos_textos.append(prompt_code_str)

# Juntar todas as strings em uma única string
texto_final = "\n".join(todos_textos)  # Usando '\n' para separar as linhas

In [ ]:
import json
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Carregar o arquivo JSON
with open('/content/seu_arquivo_filtrado.json', 'r') as file:
    dados_json = json.load(file)

# Preparar todos os textos
todos_textos = []

# Iterar sobre a lista e combinar 'prompt_code' em uma string
for item in dados_json:
    if 'prompt_code' in item:
        prompt_code_str = " | ".join(item['prompt_code'])  # Usando um delimitador ' | '
        todos_textos.append(prompt_code_str)

# Função para agrupar as sentenças
def group_sentences_into_lists(sentences, group_size):
    grouped_lists = []
    for i in range(0, len(sentences), group_size):
        grouped_lists.append(sentences[i:i+group_size])
    return grouped_lists

# Função para converter as sentenças agrupadas em um dicionário
def convert_to_dictionary(sentences, group_size):
    grouped_sentences = group_sentences_into_lists(sentences, group_size)
    dictionary = {}
    for i, group in enumerate(grouped_sentences, start=1):
        dictionary[i] = group
    return dictionary

# Função para gerar dados (sequências)
def data_generator(descriptions, tokenizer, max_length, vocab_size):
    while 1:
        for key, description_list in descriptions.items():
            input_sequence, output_word = create_sequences(tokenizer, max_length, description_list, vocab_size)
            yield input_sequence, output_word

# Função para criar as sequências
def create_sequences(tokenizer, max_length, desc_list, vocab_size):
    X1, y = list(), list()
    # Walk through each description
    for desc in desc_list:
        # Encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # Split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            # Split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # Pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # Encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # Store
            X1.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(y)

# Definindo parâmetros
group_size = 5
max_length = 38  # Exemplo de comprimento máximo, ajuste conforme necessário
vocab_size = 8829  # Número de palavras no vocabulário (ajuste conforme o seu vocabulário)

# Converter os textos em dicionário de descrições agrupadas
sentences_dictionary = convert_to_dictionary(todos_textos, group_size)

# Supondo que tokenizer já tenha sido treinado, por exemplo com `Tokenizer.fit_on_texts(todos_textos)`
# Agora podemos usar o gerador de dados

tokenizer = Tokenizer()
tokenizer.fit_on_texts(todos_textos)

# Gerador de dados para treinamento
# generator = data_generator(sentences_dictionary, tokenizer, max_length, vocab_size)

# Para pegar um lote de dados
# a, b = next(generator)
# print(a.shape, b.shape)


In [ ]:
sentences_dictionary

{1: ['P | r | o | m | p | t | : |   | W | r | i | t | e |   | a |   | f | u | n | c | t | i | o | n |   | t | o |   | f | i | n | d |   | t | h | e |   | s | h | a | r | e | d |   | e | l | e | m | e | n | t | s |   | f | r | o | m |   | t | h | e |   | g | i | v | e | n |   | t | w | o |   | l | i | s | t | s | . |   | | |   | C | o | d | e | : |   | d | e | f |   | s | i | m | i | l | a | r | _ | e | l | e | m | e | n | t | s | ( | t | e | s | t | _ | t | u | p | 1 | , |   | t | e | s | t | _ | t | u | p | 2 | ) | : | \n |   |   | r | e | s |   | = |   | t | u | p | l | e | ( | s | e | t | ( | t | e | s | t | _ | t | u | p | 1 | ) |   | & |   | s | e | t | ( | t | e | s | t | _ | t | u | p | 2 | ) | ) | \n |   |   | r | e | t | u | r | n |   | ( | r | e | s | ) |  ',
  'P | r | o | m | p | t | : |   | W | r | i | t | e |   | a |   | p | y | t | h | o | n |   | f | u | n | c | t | i | o | n |   | t | o |   | i | d | e | n | t | i | f | y |   | n | o | n | - | p | r | i | m | e |   | n

In [ ]:
from keras.layers import Dropout


def define_model(vocab_size, max_length):
    inputs1 = Input(shape=(max_length,))

    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs1)

    se2 = Dropout(0.3)(se1)

    se3 = LSTM(1024, return_sequences=True)(se2)

    se4 = Dropout(0.3)(se3)

    se5 = LSTM(1024)(se4)

    decoder1 = Dense(512, activation='relu')(se5)

    dropout1 = Dropout(0.3)(decoder1)

    decoder2 = Dense(512, activation='relu')(dropout1)

    dropout2 = Dropout(0.3)(decoder2)

    decoder3 = Dense(512, activation='relu')(dropout2)

    dropout3 = Dropout(0.3)(decoder3)

    outputs = Dense(vocab_size, activation='softmax')(dropout3)

    model = Model(inputs=[inputs1], outputs=outputs)

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    print(model.summary())

    return model

In [ ]:
model = define_model(vocab_size, max_length)

print('Descriptions: train=', len(sentences_dictionary))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)
model = define_model(vocab_size, max_length)
epochs = 10
steps = len(sentences_dictionary)
for i in range(epochs):
 generator = data_generator(sentences_dictionary, tokenizer, max_length, vocab_size)
 model.fit(generator, epochs=1, steps_per_epoch= steps, verbose=1,)
 model.save("models/textmodel_" + str(i) + ".h5")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 38)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 38, 256)        │      2,260,224 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 38, 256)        │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, 38)             │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 38, 1024)       │      5,246,976 │ dropout[0][0],         │
│                           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 38, 1024)       │              0 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 1024)           │      8,392,704 │ dropout_1[0][0],       │
│                           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 512)            │        524,800 │ lstm_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 512)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 512)            │        262,656 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 512)            │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 512)            │        262,656 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 512)            │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 8829)           │      4,529,277 │ dropout_4[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 21,479,293 (81.94 MB)

 Trainable params: 21,479,293 (81.94 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 38)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 38, 256)        │      2,260,224 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 38, 256)        │              0 │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_1 (NotEqual)    │ (None, 38)             │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ (None, 38, 1024)       │      5,246,976 │ dropout_5[0][0],       │
│                           │                        │                │ not_equal_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_6 (Dropout)       │ (None, 38, 1024)       │              0 │ lstm_2[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ (None, 1024)           │      8,392,704 │ dropout_6[0][0],       │
│                           │                        │                │ not_equal_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 512)            │        524,800 │ lstm_3[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 512)            │              0 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 512)            │        262,656 │ dropout_7[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_8 (Dropout)       │ (None, 512)            │              0 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 512)            │        262,656 │ dropout_8[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_9 (Dropout)       │ (None, 512)            │              0 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 8829)           │      4,529,277 │ dropout_9[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 21,479,293 (81.94 MB)

 Trainable params: 21,479,293 (81.94 MB)

 Non-trainable params: 0 (0.00 B)

None
63/86 ━━━━━━━━━━━━━━━━━━━━ 21:25 56s/step - loss: 4.7970

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


In [ ]:
import tensorflow as tf
from transformers import T5Tokenizer

model = tf.keras.models.load_model("/content/textmodel_4.h5")

tokenizer = T5Tokenizer.from_pretrained("t5-small")

print(model.summary())

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/textmodel_4.h5')
predict_next_words= 30

input_text = "a man"

for _ in range(predict_next_words):
    token_list = tokenizer.texts_to_sequences([input_text])[0]
    print(token_list)
    token_list = pad_sequences([token_list], maxlen=max_length, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            if word == "end":  # Check if the predicted word is the end token
                break
            output_word = word
            break
    input_text += " " + output_word

print(input_text)